In [1]:
%%capture
# !pip install dkpro-cassis pdfplumber PyMuPDF stanza

In [2]:
# !pip install torch tensorflow sentencepiece transformers


This can be used as A global function to extract both the exel data and the pdfs using the csv files 


In [3]:
import pandas as pd

file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
data_id = pd.read_excel(file_path_excel, usecols=["Kriterium-ID"])["Kriterium-ID"].tolist()
data = pd.read_excel(file_path_excel, usecols=["Kategorie Kriterium", "Ausschreibungskriterium"])
texts = data.apply(lambda row: " ".join(row.dropna().astype(str)), axis=1).tolist()
# Step 2: Load sentences from CSV if provided


# File paths
file_paths = [
    "DATA/1_Harmonisiert_pflichtenheft.csv",
    "DATA/2_Harmonisiert_pflichtenheft.csv",
    "DATA/4_Harmonisiert_ausschreibung.csv"
]

# Load and combine all CSV files into a single DataFrame
dataframes = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(dataframes, ignore_index=True)

# Extract sentences
sentences = combined_df["text"].tolist()

This is an example of how to use these data and an example of NLI method as well 

In [ ]:
import pandas as pd
from transformers import pipeline

# Load German NLI model
nli_pipeline = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
)
results = []
for text_index, text in enumerate(texts[:5]):
    for sentence in sentences[:5]:  # Limit to the first 5 sentences
        # Run NLI for the sentence against the current text
        result = nli_pipeline(sentence, [text], multi_label=True)
        score = result["scores"][0]  # Extract score for the single label
        results.append({"sentence": sentence, "text_index": text_index, "text": text, "score": score})
print(f"Computed similarity scores for {len(results)} sentence-text pairs.")

# Step 4: Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Step 5: Extract the top 20% matches for each sentence
top_20_percent = results_df.groupby("sentence").apply(
    lambda group: group.nlargest(max(1, len(group) // 5), "score")
).reset_index(drop=True)

print(f"Extracted top 20% matches for each sentence.")

# Step 6: Save the results to a CSV file
output_file = "top_20_percent_matches.csv"
top_20_percent.to_csv(output_file, index=False)
print(f"Top 20% matches saved to '{output_file}'.")

/home/fcr/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-08 20:56:27.374062: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 20:56:27.380708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733687787.388850   84793 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733687787.391251   84793 cuda_blas.cc:1418] Unable to reg

Computed similarity scores for 25 sentence-text pairs.
Extracted top 20% matches for each sentence.
Top 20% matches saved to 'top_20_percent_matches.csv'.


/tmp/ipykernel_84793/2099341049.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_20_percent = results_df.groupby("sentence").apply(


In [6]:
print(top_20_percent)

                                            sentence  text_index  \
0  4 Verordnung über das öffentliche Beschaffungs...           1   
1  BSG 731.21) Wichtig Die Implementierung des Re...           1   
2  Der politische Prozess hat gezeigt, dass sowoh...           0   
3  Deshalb wird dieses Geschäft «unter Vorbehalt»...           3   
4  Öffentliche Submission Gesunde Ernährung «Kalt...           0   

                                                text     score  
0  TS Mindestens ein vegetarisches oder veganes G...  0.522896  
1  TS Mindestens ein vegetarisches oder veganes G...  0.470789  
2  TS Das Angebot umfasst eine attraktive Auswahl...  0.311388  
3  ZK Mindestens 50% des Tagesangebots sind veget...  0.311152  
4  TS Das Angebot umfasst eine attraktive Auswahl...  0.693403  
